<h1><center>SpaGCN Tutorial Easy Mode</center></h1>


<center>Author: Jian Hu,*, Xiangjie Li, Kyle Coleman, Amelia Schroeder, Nan Ma, David J. Irwin, Edward B. Lee, Russell T. Shinohara, Mingyao Li*

### Outline
1. Installation
2. Import modules
3. Read in data
4. Spatial domain detection using SpaGCN
5. Identify SVGs
6. Identify Meta Gene

### 1. Installation
To install SpaGCN package you must make sure that your python version is over 3.5.=. If you don’t know the version of python you can check it by:

In [1]:
import platform
platform.python_version()

'3.8.8'

Note: Because SpaGCN pends on pytorch, you should make sure torch is correctly installed.
<br>
Now you can install the current release of SpaGCN by the following three ways:
#### 1.1 PyPI: Directly install the package from PyPI.

In [ ]:
pip3 install SpaGCN
#Note: you need to make sure that the pip is for python3，or we should install SpaGCN by
python3 -m pip install SpaGCN
pip3 install SpaGCN
#If you do not have permission (when you get a permission denied error), you should install SpaGCN by
pip3 install --user SpaGCN

#### 1.2 Github
Download the package from Github and install it locally:

In [2]:
git clone https://github.com/jianhuupenn/SpaGCN
cd SpaGCN/SpaGCN_package/
python3 setup.py install --user

#### 1.3 Anaconda
If you do not have Python3.5 or Python3.6 installed, consider installing Anaconda (see Installing Anaconda). After installing Anaconda, you can create a new environment, for example, SpaGCN (you can change to any name you like).

In [ ]:
#create an environment called SpaGCN
conda create -n SpaGCN python=3.7.9
#activate your environment 
conda activate SpaGCN
git clone https://github.com/jianhuupenn/SpaGCN
cd SpaGCN/SpaGCN_package/
python3 setup.py build
python3 setup.py install
conda deactivate

### 2. Import python modules

In [27]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2

adata=sc.read("../tutorial/data/151673/sample_data.h5ad")
#Read in hitology image
#adata=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp.h5ad")

img=cv2.imread("../tutorial/data/151673/histology.tif")
#img = cv2.imread("/home/qyyuan/project/ST_GRN/data/SCC/GSM4284316_P2_ST_rep1.jpg")
print(adata)

print(adata.obs["x2"])
print(adata.obs["x3"])
print(adata.obs["x4"])
print(adata.obs["x5"])
#Set coordinates
adata.obs["x_array"]=adata.obs["x2"]
adata.obs["y_array"]=adata.obs["x3"]
adata.obs["x_pixel"]=adata.obs["x4"]
adata.obs["y_pixel"]=adata.obs["x5"]
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()
#Run SpaGCN
adata.obs["pred"]= spg.detect_spatial_domains_ez_mode(adata, img, x_array, y_array, x_pixel, y_pixel, n_clusters=7, histology=True, s=1, b=49, p=0.5, r_seed=100, t_seed=100, n_seed=100)
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Refine domains (optional)
#shape="hexagon" for Visium data, "square" for ST data.
adata.obs["refined_pred"]=spg.spatial_domains_refinement_ez_mode(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), x_array=x_array, y_array=y_array, shape="hexagon")
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')


plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/pred.png")
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="refined_pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/refined_pred.png")





target=0
#Read in raw data
#raw=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp.h5ad")
raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
#Preprocessing
raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x2"]
raw.obs["y_array"]=raw.obs["x3"]
raw.obs["x_pixel"]=raw.obs["x4"]
raw.obs["y_pixel"]=raw.obs["x5"]
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)
#Set filtering criterials





AnnData object with n_obs × n_vars = 3639 × 33538
    obs: 'x1', 'x2', 'x3', 'x4', 'x5'
    var: 'gene_ids', 'feature_types', 'genome', 'genename'
AAACAAGTATCTCCCA-1    50
AAACAATCTACTAGCA-1     3
AAACACCAATAACTGC-1    59
AAACAGAGCGACTCCT-1    14
AAACAGCTTTCAGAAG-1    43
                      ..
TTGTTTCACATCCAGG-1    58
TTGTTTCATTAGTCTA-1    60
TTGTTTCCATACAACT-1    45
TTGTTTGTATTACACG-1    73
TTGTTTGTGTAAATTC-1     7
Name: x2, Length: 3639, dtype: int64
AAACAAGTATCTCCCA-1    102
AAACAATCTACTAGCA-1     43
AAACACCAATAACTGC-1     19
AAACAGAGCGACTCCT-1     94
AAACAGCTTTCAGAAG-1      9
                     ... 
TTGTTTCACATCCAGG-1     42
TTGTTTCATTAGTCTA-1     30
TTGTTTCCATACAACT-1     27
TTGTTTGTATTACACG-1     41
TTGTTTGTGTAAATTC-1     51
Name: x3, Length: 3639, dtype: int64
AAACAAGTATCTCCCA-1     8468
AAACAATCTACTAGCA-1     2807
AAACACCAATAACTGC-1     9505
AAACAGAGCGACTCCT-1     4151
AAACAGCTTTCAGAAG-1     7583
                      ...  
TTGTTTCACATCCAGG-1     9396
TTGTTTCATTAGTCTA-1    

In [28]:
raw.obs["pred"].unique()

[3, 1, 5, 2, 4, 6, 0]
Categories (7, int64): [0, 1, 2, 3, 4, 5, 6]

In [29]:
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.2
all_filtered = []
for i in range(7):
    filtered_info=spg.detect_SVGs_ez_mode(raw, target=i, x_name="x_array", y_name="y_array", domain_name="pred", min_in_group_fraction=min_in_group_fraction, min_in_out_group_ratio=min_in_out_group_ratio, min_fold_change=min_fold_change)
    all_filtered.append(filtered_info)




Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.970561981201172], num_nbr [1.0, 411.82522123893807]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.192387759685516], num_nbr [1.0, 132.57743362831857]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.303300648927689], num_nbr [1.0, 44.20132743362832]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3587570935487747], num_nbr [1.0, 20.754424778761063]
Calculateing adj matrix using xy only...
Run 5: radius [2.386485315859318, 3.3587570935487747], num_nbr [8.938053097345133, 20.754424778761063]
Calculateing adj matrix using xy only...
recommended radius =  2.8726212047040462 num_nbr=12.882743362831858
radius= 2.8726212047040462 average number of neighbors for each spot is 12.882743362831858
 Cluster 0 has neighbors:
Dmain  4 :  931
Dmain  3 : 

In [30]:
final_df = pd.concat(all_filtered, ignore_index=True)



In [31]:
final_df.shape

(130, 10)

## SCC

In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg

import cv2
#adata=sc.read("../tutorial/data/151673/sample_data.h5ad")
#Read in hitology image
adata=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp.h5ad")
#img=cv2.imread("../tutorial/data/151673/histology.tif")
img = cv2.imread("/home/qyyuan/project/ST_GRN/data/SCC/GSM4284316_P2_ST_rep1.jpg")
#adata=adata[adata.obs['x']>=3282,:]
#adata=adata[adata.obs['x']<=11044,:]
#adata=adata[adata.obs['y']>=2943,:]
#adata=adata[adata.obs['y']<=11745,:]
# 生成从0到19691中随机且不重复的5000个索引
random_indices = np.random.choice(adata.shape[0], size=5000, replace=False)
adata=adata[random_indices,:]
#Set coordinates
adata.obs["x_array"]=adata.obs["x"]
adata.obs["y_array"]=adata.obs["y"]
adata.obs["x_pixel"]=adata.obs["x"]
adata.obs["y_pixel"]=adata.obs["y"]
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()
#Run SpaGCN
adata.obs["pred"]= spg.detect_spatial_domains_ez_mode(adata, img, x_array, y_array, x_pixel, y_pixel, n_clusters=8, histology=False, s=1, b=49, p=0.5, r_seed=100, t_seed=100, n_seed=100)
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Refine domains (optional)
#shape="hexagon" for Visium data, "square" for ST data.
adata.obs["refined_pred"]=spg.spatial_domains_refinement_ez_mode(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), x_array=x_array, y_array=y_array, shape="hexagon")
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/pred_scc.png")
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="refined_pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/refined_pred_scc.png")


/home/zzhang/.conda/envs/SpaGCN/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 482.7071498817345]
Run 2: l [0.01, 500.005], p [0.0, 134.8173370361328]
Run 3: l [0.01, 250.0075], p [0.0, 35.29075622558594]
Run 4: l [0.01, 125.00874999999999], p [0.0, 8.847901344299316]
Run 5: l [0.01, 62.509375], p [0.0, 2.0600764751434326]
Run 6: l [31.2596875, 62.509375], p [0.34165072441101074, 2.0600764751434326]
Run 7: l [31.2596875, 46.884531249999995], p [0.34165072441101074, 1.0590918064117432]
Run 8: l [31.2596875, 39.072109375], p [0.34165072441101074, 0.6642723083496094]
recommended l =  35.1658984375
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  23
Initializing cluster centers with louvain, resolution =  0.6
Epoch  0
Epoch  10
Res =  0.6 Num of clusters =  20
Res changed to 0.6
Initializing cluster centers with louvain, resolution =  0.5
Epoch  0
Epoch  10
Res =  0.5 Num of clusters =  16
Res changed to 

In [4]:
adata.obs.to_csv('/home/zzhang/SPADE/SpaGCN/tutorial/sample_results/scc_domian_6.txt',sep='\t')

In [34]:
adata.obs["pred"].unique()

[6, 4, 0, 7, 3, 1, 5, 2, 8, 9]
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9]

In [ ]:
target=0
#Read in raw data
raw=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp.h5ad")
#raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
# Preprocessing
raw=raw[random_indices,:]
# raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x"]
raw.obs["y_array"]=raw.obs["y"]
raw.obs["x_pixel"]=raw.obs["x"]
raw.obs["y_pixel"]=raw.obs["y"]
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1

raw.X=np.array(raw.X,dtype='float32')


In [43]:
all_filtered = []  # 用 list 存储每轮结果
min_fold_change=1.4
for i in range(10):
    filtered_info=spg.detect_SVGs_ez_mode(raw, target=i, x_name="x_array", y_name="y_array", domain_name="pred", min_in_group_fraction=min_in_group_fraction, min_in_out_group_ratio=min_in_out_group_ratio, min_fold_change=min_fold_change)
    all_filtered.append(filtered_info)

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [141.42135620117188, 1392.8388671875], num_nbr [5.941908713692946, 495.7939142461964]
Calculateing adj matrix using xy only...
Run 2: radius [141.42135620117188, 767.1301116943359], num_nbr [5.941908713692946, 166.9875518672199]
Calculateing adj matrix using xy only...
Run 3: radius [141.42135620117188, 454.2757339477539], num_nbr [5.941908713692946, 60.95850622406639]
Calculateing adj matrix using xy only...
Run 4: radius [141.42135620117188, 297.8485450744629], num_nbr [5.941908713692946, 26.575380359612726]
Calculateing adj matrix using xy only...
Run 5: radius [141.42135620117188, 219.63495063781738], num_nbr [5.941908713692946, 15.4716459197787]
Calculateing adj matrix using xy only...
recommended radius =  180.52815341949463 num_nbr=11.61549100968188
radius= 180.52815341949463 average number of neighbors for each spot is 11.61549100968188
 Clust

In [45]:
final_df = pd.concat(all_filtered, ignore_index=True)

In [46]:
final_df.shape

(247, 10)

In [47]:
# Save to CSV
final_df.to_csv('svg-enhanced-scc-new-0830.csv', index=False)

### Mouse Brain

In [48]:
"""
#Read original 10x_h5 data and save it to h5ad
from scanpy import read_10x_h5
adata = read_10x_h5("../tutorial/data/151673/expression_matrix.h5")
spatial=pd.read_csv("../tutorial/data/151673/positions.txt",sep=",",header=None,na_filter=False,index_col=0) 
adata.obs["x1"]=spatial[1]
adata.obs["x2"]=spatial[2]
adata.obs["x3"]=spatial[3]
adata.obs["x4"]=spatial[4]
adata.obs["x5"]=spatial[5]
#Select captured samples
adata=adata[adata.obs["x1"]==1]
adata.var_names=[i.upper() for i in list(adata.var_names)]
adata.var["genename"]=adata.var.index.astype("str")
adata.write_h5ad("../tutorial/data/151673/sample_data.h5ad")
"""
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2
#Read in gene expression and spatial location
adata=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp1.h5ad")
#Read in hitology image
img=cv2.imread("/home/qyyuan/project/ST_GRN/data/mouse_anterior/V1_Mouse_Brain_Sagittal_Anterior_image.tif")

In [50]:
# 生成从0到19691中随机且不重复的5000个索引
random_indices = np.random.choice(adata.shape[0], size=5000, replace=False)
adata=adata[random_indices,:]
#Set coordinates
adata.obs["x_array"]=adata.obs["x"]
adata.obs["y_array"]=adata.obs["y"]
adata.obs["x_pixel"]=adata.obs["x"]
adata.obs["y_pixel"]=adata.obs["y"]
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()
#Run SpaGCN
adata.obs["pred"]= spg.detect_spatial_domains_ez_mode(adata, img, x_array, y_array, x_pixel, y_pixel, n_clusters=15, histology=False, s=1, b=49, p=0.5, r_seed=100, t_seed=100, n_seed=100)
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Refine domains (optional)
#shape="hexagon" for Visium data, "square" for ST data.
adata.obs["refined_pred"]=spg.spatial_domains_refinement_ez_mode(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), x_array=x_array, y_array=y_array, shape="hexagon")
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/pred1.png")
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="refined_pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/refined_pred1.png")


Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 493.25897752819174]
Run 2: l [0.01, 500.005], p [0.0, 140.92559814453125]
Run 3: l [0.01, 250.0075], p [0.0, 37.44088363647461]
Run 4: l [0.01, 125.00874999999999], p [0.0, 9.490480422973633]
Run 5: l [0.01, 62.509375], p [0.0, 2.231536626815796]
Run 6: l [31.2596875, 62.509375], p [0.3745049238204956, 2.231536626815796]
Run 7: l [31.2596875, 46.884531249999995], p [0.3745049238204956, 1.1521644592285156]
Run 8: l [31.2596875, 39.072109375], p [0.3745049238204956, 0.7249996662139893]
Run 9: l [31.2596875, 35.1658984375], p [0.3745049238204956, 0.5398757457733154]
Run 10: l [33.212792968749994, 35.1658984375], p [0.45462751388549805, 0.5398757457733154]
recommended l =  34.189345703125
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  21
Initializing cluster centers with louvain, resolution =  0.6
Epoch  0
Epoch  10
Res =  0.

In [51]:

raw=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp1.h5ad")
#raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
# Preprocessing
raw=raw[random_indices,:]
# raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x"]
raw.obs["y_array"]=raw.obs["y"]
raw.obs["x_pixel"]=raw.obs["x"]
raw.obs["y_pixel"]=raw.obs["y"]
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)
#Set filtering criterials


In [52]:
raw.obs["pred"].unique()

[5, 0, 1, 14, 8, ..., 9, 6, 13, 11, 12]
Length: 15
Categories (15, int64): [0, 1, 2, 3, ..., 11, 12, 13, 14]

In [53]:
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
raw.X=np.array(raw.X,dtype='float32')
all_filtered = []
for i in range(15):
    filtered_info=spg.detect_SVGs_ez_mode(raw, target=0, x_name="x_array", y_name="y_array", domain_name="pred", min_in_group_fraction=min_in_group_fraction, min_in_out_group_ratio=min_in_out_group_ratio, min_fold_change=min_fold_change)
    all_filtered.append(filtered_info)



Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [111.80339813232422, 1372.9530029296875], num_nbr [4.173357664233577, 470.7810218978102]
Calculateing adj matrix using xy only...
Run 2: radius [111.80339813232422, 742.3782005310059], num_nbr [4.173357664233577, 160.50547445255475]
Calculateing adj matrix using xy only...
Run 3: radius [111.80339813232422, 427.09079933166504], num_nbr [4.173357664233577, 57.2992700729927]
Calculateing adj matrix using xy only...
Run 4: radius [111.80339813232422, 269.44709873199463], num_nbr [4.173357664233577, 25.92153284671533]
Calculateing adj matrix using xy only...
recommended radius =  190.62524843215942 num_nbr=12.578467153284672
radius= 190.62524843215942 average number of neighbors for each spot is 12.578467153284672
 Cluster 0 has neighbors:
Dmain  14 :  276
SVGs for domain  0 : ['Rnf103', 'Fbxo10', 'Pde1a', 'Coro7', 'Hjurp', 'Cadps2', 'Ttc33', 'Grin3a', 'A

In [54]:
final_df = pd.concat(all_filtered, ignore_index=True)



In [55]:
final_df.shape

(255, 10)

In [56]:
# Save to CSV
df.to_csv('svg-enhanced-mouse-new.csv', index=False)

### PDAC

In [14]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
import cv2
adata=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp_PDAC.h5ad")
try:
    img = cv2.imread("/home/qyyuan/project/ST_GRN/data/PDAC/GSM3036911_PDAC-A-ST1-HE.jpg", 
                     cv2.IMREAD_IGNORE_ORIENTATION | cv2.IMREAD_COLOR)
    if img is not None:
        print(f"方法1成功，尺寸: {img.shape}")
    else:
        raise Exception("读取失败")
except:
    try:
        img = cv2.imread("/home/qyyuan/project/ST_GRN/data/PDAC/GSM3036911_PDAC-A-ST1-HE.jpg", 
                         cv2.IMREAD_REDUCED_COLOR_4)  # 1/4尺寸
        if img is not None:
            print(f"方法2成功（缩小版），尺寸: {img.shape}")
    except Exception as e:
        print(f"OpenCV方法都失败: {e}")


# Set coordinates
adata.obs["x_array"]=adata.obs["x"]
adata.obs["y_array"]=adata.obs["y"]
adata.obs["x_pixel"]=adata.obs["x"]
adata.obs["y_pixel"]=adata.obs["y"]
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()
#Run SpaGCN
adata.obs["pred"]= spg.detect_spatial_domains_ez_mode(adata, img, x_array, y_array, x_pixel, y_pixel, n_clusters=10, histology=False, s=1, b=49, p=0.5, r_seed=100, t_seed=100, n_seed=100)
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Refine domains (optional)
#shape="hexagon" for Visium data, "square" for ST data.
adata.obs["refined_pred"]=spg.spatial_domains_refinement_ez_mode(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), x_array=x_array, y_array=y_array, shape="hexagon")
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/pred_PDAC.png")
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="refined_pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/refined_pred_PDAC.png")


方法2成功（缩小版），尺寸: (9007, 10578, 3)
Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 1626.1557363858617]
Run 2: l [0.01, 500.005], p [0.0, 507.9877014160156]
Run 3: l [0.01, 250.0075], p [0.0, 140.65602111816406]
Run 4: l [0.01, 125.00874999999999], p [0.0, 36.3209114074707]
Run 5: l [0.01, 62.509375], p [0.0, 8.580265045166016]
Run 6: l [0.01, 31.2596875], p [0.0, 1.4352269172668457]
Run 7: l [15.63484375, 31.2596875], p [0.023842215538024902, 1.4352269172668457]
Run 8: l [23.447265625, 31.2596875], p [0.44752657413482666, 1.4352269172668457]
Run 9: l [23.447265625, 27.3534765625], p [0.44752657413482666, 0.8862314224243164]
Run 10: l [23.447265625, 25.40037109375], p [0.44752657413482666, 0.6509405374526978]
Run 11: l [23.447265625, 24.423818359374998], p [0.44752657413482666, 0.544939398765564]
recommended l =  23.9355419921875
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters

In [15]:
target=0
#Read in raw data
raw=sc.read("/home/qyyuan/project/ST_GRN/TESLA/tutorial/results/enhanced_exp_PDAC.h5ad")
#raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
# Preprocessing
raw=raw[random_indices,:]
# raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x"]
raw.obs["y_array"]=raw.obs["y"]
raw.obs["x_pixel"]=raw.obs["x"]
raw.obs["y_pixel"]=raw.obs["y"]
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)
#Set filtering criterials

In [23]:
min_in_group_fraction=0.7
min_in_out_group_ratio=1
min_fold_change=1.4
raw.X=np.array(raw.X,dtype='float32')
all_filtered = [] 
for i in range(10):
    filtered_info=spg.detect_SVGs_ez_mode(raw, target=i, x_name="x_array", y_name="y_array", domain_name="pred", min_in_group_fraction=min_in_group_fraction, min_in_out_group_ratio=min_in_out_group_ratio, min_fold_change=min_fold_change)
    all_filtered.append(filtered_info)
    print(filtered_info.shape)
final_df = pd.concat(all_filtered, ignore_index=True)


Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [100.0, 894.4271850585938], num_nbr [8.513542795232937, 574.2394366197183]
Calculateing adj matrix using xy only...
Run 2: radius [100.0, 497.2135925292969], num_nbr [8.513542795232937, 187.2697724810401]
Calculateing adj matrix using xy only...
Run 3: radius [100.0, 298.60679626464844], num_nbr [8.513542795232937, 68.17551462621886]
Calculateing adj matrix using xy only...
Run 4: radius [100.0, 199.30339813232422], num_nbr [8.513542795232937, 28.508125677139763]
Calculateing adj matrix using xy only...
Run 5: radius [100.0, 149.6516990661621], num_nbr [8.513542795232937, 16.032502708559047]
Calculateing adj matrix using xy only...
recommended radius =  124.82584953308105 num_nbr=13.540628385698808
radius= 124.82584953308105 average number of neighbors for each spot is 13.540628385698808
 Cluster 0 has neighbors:
Dmain  7 :  245
Dmain  6 :  199
SVGs f

In [24]:
final_df.shape

(158, 10)

In [25]:

# Save to CSV
final_df.to_csv('svg-enhanced-PDAC.csv', index=False)

#### Plot spatial domains

In [7]:
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/pred.png")
ax=spg.plot_spatial_domains_ez_mode(adata, domain_name="refined_pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/refined_pred.png")

**Spatial Domains**![](./sample_results/pred.png) **Refined Spatial Domains**![](./sample_results/refined_pred.png)

### 5. Identify SVGs

- target: Target domain to identify SVGs.
- min_in_group_fraction: Minium in-group expression fraction.
- min_in_out_group_ratio: Miniumn (in-group expression fraction) / (out-group expression fraction).
- min_fold_change: Miniumn (in-group expression) / (out-group expression).
- r: Radius to detect a spot's neighboring spots.target: Target domain to identify SVGs.

In [ ]:
target=0
#Read in raw data
adata=sc.read("../tutorial/sample_results/results.h5ad")
raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
#Preprocessing
raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x2"]
raw.obs["y_array"]=raw.obs["x3"]
raw.obs["x_pixel"]=raw.obs["x4"]
raw.obs["y_pixel"]=raw.obs["x5"]
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
filtered_info=spg.detect_SVGs_ez_mode(raw, target=0, x_name="x_array", y_name="y_array", domain_name="pred", min_in_group_fraction=min_in_group_fraction, min_in_out_group_ratio=min_in_out_group_ratio, min_fold_change=min_fold_change)

#### Plot

In [16]:
color_self = clr.LinearSegmentedColormap.from_list('pink_green', ['#3AB370',"#EAE7CC","#FD1593"], N=256)
spg.plot_SVGs_ez_mode(adata=raw, gene_list=filtered_info["genes"].tolist(), x_name="y_pixel", y_name="x_pixel", plot_color=color_self, size=50, show=False, save=True, save_dir="./sample_results/")

**CAMK2N1**![](./sample_results/CAMK2N1.png) **ENC1**![](./sample_results/ENC1.png) **GPM6A**![](./sample_results/GPM6A.png) **ARPP19**![](c/ARPP19.png) **HPCAL1**![](./sample_results/HPCAL1.png)

### 6. Identify Meta Gene

- target: Target domain to detect meta gene.
- start_gene: Gene to satrt with. Candidate start genes can be found by lowering the filtering criterials.

In [ ]:
target=2
raw.obs["meta"]=spg.detect_meta_genes_ez_mode(raw, target, x_name="x_array", y_name="y_array", domain_name="pred", start_gene="GFAP", use_raw=False)

#### Plot

In [18]:
spg.plot_meta_genes_ez_mode(raw, x_name="y_pixel", y_name="x_pixel", meta_name="meta", plot_color=color_self, size=50, show=False, save=True, save_dir="./")

**start**![](./sample_results/GFAP.png) **meta gene**![](./sample_results/meta_gene.png)